In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import pyodbc
from fake_useragent import UserAgent
import random

# URL de la sección de componentes de PC
section_url = "https://listado.mercadolibre.com.ar/componentes-de-pc#D[A:Componentes%20de%20PC]" 



# Configuración avanzada
ua = UserAgent(
    browsers=['chrome', 'firefox', 'safari'],  # Navegadores comunes
    platforms=['windows', 'macos', 'linux'],   # Sistemas operativos
    min_percentage=1.5  # Solo agentes con al menos 1.5% de uso global
)

def get_random_headers():
    return {
        'User-Agent': ua.random,
        'Accept-Language': 'es-ES,es;q=0.9',
        'Referer': random.choice([
            'https://www.google.com/',
            'https://www.mercadolibre.com.ar/',
            'https://www.bing.com/'
        ]),
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'DNT': str(random.randint(1, 2))  # 30% de probabilidad de DNT
    }
    



# Configuración de threading
MAX_THREADS = 10  # Ajustar según necesidad y para no saturar el servidor
REQUEST_TIMEOUT = 10  # Segundos

# Configuración de SQL Server
SERVER = 'localhost'  # Ejemplo: 'LAPTOP-ABC123\SQLEXPRESS' o 'localhost'
DATABASE = 'MercadoLibre'
USERNAME = 'mateoYo'     # Dejar en blanco si usas autenticación de Windows
PASSWORD = 'gatogris'  # Dejar en blanco si usas autenticación de Windows
TRUSTED_CONNECTION = 'yes'  # Usar 'yes' para autenticación de Windows, 'no' para SQL auth

def get_product_links(base_url, num_pages=5):
    product_links = []
    for i in range(num_pages):
        page_offset = i * 48
        url = f"{base_url}_Desde_{page_offset}_NoIndex_True" if page_offset > 0 else base_url

        try:
            response = requests.get(url, headers=get_random_headers(), timeout=REQUEST_TIMEOUT)
            soup = BeautifulSoup(response.content, "html.parser")

            for a_tag in soup.find_all("a", href=True):
                href = a_tag["href"]
                if "mercadolibre.com.ar" in href and "/p/" in href:
                    full_link = href.split("?")[0]
                    if full_link not in product_links:
                        product_links.append(full_link)
        except Exception as e:
            print(f"Error obteniendo enlaces de página {i+1}: {e}")

    return product_links

#Clasificación optimizada con diccionario
PRODUCT_CATEGORIES = {
    'procesador': ['procesador', 'cpu', 'core i'],
    'ram': ['ram', 'memoria'],
    'placa de video': ['placa de video', 'gpu', 'rtx', 'gtx'],
    'disco': ['disco', 'ssd', 'hdd'],
    'gabinete': ['gabinete'],
    'placa madre': ['placa madre', 'motherboard'],
    'cooler': ['cooler', 'ventilador'],
    'pasta térmica': ['térmica', 'thermal']
}

def classify_product_type(title):
    """Clasificación con búsqueda por palabras clave"""
    if not title:
        return "Otro"
    
    title_lower = title.lower()
    for category, keywords in PRODUCT_CATEGORIES.items():
        if any(keyword in title_lower for keyword in keywords):
            return category
    return "Otro"

def extract_product_data(URL):
    """
    recibe un link, extrae los datos utilizando requests y BeautifulSoup con las etiquetas HTLM correspondientes y crea un diccionario con las claves; Título, tipo, Precio, Calificación, Opiniones, Vendidos
    :parametro (URL): URl de un producto especifico
    :return: diccionario datos estruturados
    """
    product_data = {'URL': URL}
    
    try:
        webpage = requests.get(URL, headers=get_random_headers(), timeout=REQUEST_TIMEOUT)
        soup = BeautifulSoup(webpage.content, "html.parser")

        # Título y tipo
        try:
            title = soup.find("h1", class_="ui-pdp-title").text.strip()
            product_data['Title'] = title
            product_data['Type'] = classify_product_type(title)
        except AttributeError:
            product_data['Title'] = "NA"
            product_data['Type'] = "NA"

        # Precio
        try:
            price = soup.find("span", class_="andes-money-amount__fraction").text.strip().replace(".", "")
            product_data['Price'] = int(price)
        except (AttributeError, ValueError):
            product_data['Price'] = 0  # Usar 0 para valor numérico

        # Calificación
        try:
            rating = soup.find("span", class_="ui-pdp-review__rating").text.strip().replace(".", ",")
            product_data['Rating'] = rating
        except AttributeError:
            product_data['Rating'] = "0"

        # Opiniones
        try:
            reviews = soup.find("span", class_="ui-pdp-review__amount").text.strip("()").replace(".", "")
            product_data['Reviews'] = int(reviews)
        except (AttributeError, ValueError):
            product_data['Reviews'] = 0  # Usar 0 para valor numérico

        # Vendidos
        try:
            sold_text = soup.find("span", class_="ui-pdp-subtitle").text.strip()
            if "vendidos" in sold_text:
                sold_number = (
                    sold_text.split("|")[-1]
                    .replace("vendidos", "")
                    .replace("+", "")
                    .replace("mil", "000")
                    .strip()
                )
                product_data['Sold'] = int(sold_number)
            else:
                product_data['Sold'] = 0  # Usar 0 para valor numérico
        except (AttributeError, ValueError):
            product_data['Sold'] = 0  # Usar 0 para valor numérico

    except Exception as e:
        print(f"Error procesando {URL}: {e}")

    return product_data

def create_db_connection():
    """Establece conexión con SQL Server"""
    conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};'
    
    # Determinar tipo de autenticación
    if TRUSTED_CONNECTION == 'yes':
        conn_str += 'Trusted_Connection=yes;'
    else:
        conn_str += f'UID={USERNAME};PWD={PASSWORD};'
    
    try:
        return pyodbc.connect(conn_str)
    except Exception as e:
        print(f"Error de conexión a la base de datos: {e}")
        return None

def create_products_table(conn):
    """Crea la tabla de productos si no existe"""
    cursor = conn.cursor()
    try:
        cursor.execute('''
        IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'ProductosMercadoLibre')
        BEGIN
            CREATE TABLE ProductosMercadoLibre (
                ID INT IDENTITY(1,1) PRIMARY KEY,
                URL NVARCHAR(500) NOT NULL,
                Title NVARCHAR(500),
                Type NVARCHAR(100),
                Price INT,
                Rating NVARCHAR(10),
                Reviews INT,
                Sold INT,
                FechaExtraccion DATETIME DEFAULT GETDATE()
            )
        END
        ''')
        conn.commit()
        print("Tabla de productos verificada/creada correctamente.")
    except Exception as e:
        print(f"Error al crear la tabla: {e}")
    finally:
        cursor.close()

def insert_product_into_db(conn, product_data):
    """Inserta un producto en la base de datos"""
    cursor = conn.cursor()
    try:
        cursor.execute('''
        INSERT INTO ProductosMercadoLibre (URL, Title, Type, Price, Rating, Reviews, Sold)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', 
        product_data['URL'], 
        product_data['Title'], 
        product_data['Type'], 
        product_data['Price'], 
        product_data['Rating'], 
        product_data['Reviews'], 
        product_data['Sold'])
        conn.commit()
        return True
    except Exception as e:
        print(f"Error al insertar producto: {e}")
        return False
    finally:
        cursor.close()

def main():
    start_time = time.time()
    
    # Conexión a la base de datos
    conn = create_db_connection()
    if not conn:
        print("No se pudo establecer conexión a la base de datos. Saliendo...")
        return
    
    # Crear tabla si no existe
    create_products_table(conn)
    
    # Obtener enlaces
    num_pages = 10
    print(f"Obteniendo enlaces de {num_pages} páginas...")
    product_links = get_product_links(section_url, num_pages=num_pages)
    print(f"Se encontraron {len(product_links)} enlaces únicos.")
    
    # Procesar productos con ThreadPoolExecutor
    print("Extrayendo datos de productos...")
    productos_procesados = 0
    productos_insertados = 0
    
    with ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
        # Crear un futuro por cada URL
        futures = {executor.submit(extract_product_data, link): link for link in product_links}
        
        # Procesar resultados conforme van terminando
        for i, future in enumerate(as_completed(futures), 1):
            link = futures[future]
            try:
                product_data = future.result()
                productos_procesados += 1
                
                # Insertar en la base de datos
                if insert_product_into_db(conn, product_data):
                    productos_insertados += 1
                
                if i % 10 == 0:  # Mostrar progreso cada 10 productos
                    print(f"Procesados {i}/{len(product_links)} productos")
            except Exception as e:
                print(f"Error procesando {link}: {e}")

    # Cerrar conexión a la base de datos
    conn.close()
    
    total_time = time.time() - start_time
    print(f"\nDatos guardados en SQL Server: {productos_insertados} productos.")
    print(f"Tiempo total: {total_time:.2f} segundos")
    print(f"Productos procesados por segundo: {productos_procesados/total_time:.2f}")

if __name__ == '__main__':
    main()

Error occurred during getting browser(s): random, but was suppressed with fallback.


Tabla de productos verificada/creada correctamente.
Obteniendo enlaces de 10 páginas...


Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fa

Se encontraron 489 enlaces únicos.
Extrayendo datos de productos...


Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.


...

Procesados 460/489 productos


Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.


Procesados 470/489 productos


Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.
Error occurred during getting browser(s): random, but was suppressed with fallback.


Procesados 480/489 productos

Datos guardados en SQL Server: 489 productos.
Tiempo total: 74.46 segundos
Productos procesados por segundo: 6.57


In [ ]:
response = requests.get(
    url, 
    params={"q": "python", "page": 2},   # (1) Parámetros de consulta (= ?q=python&page=2)
    headers=headers,                     # (2) Cabeceras HTTP personalizadas
    cookies={"sessionid": "abc123"},     # (3) Cookies a enviar
    proxies=proxies,                     # (4) Proxies HTTP/HTTPS
    timeout=10,                          # (5) Tiempo máximo en segundos
    allow_redirects=True,                # (6) Si sigue redirecciones (default: True)
    stream=False                         # (7) Si descarga de forma “streaming”
)

